### 데이터를 효과적으로 전달하는 방법

- `RunnablePassthrough()`: 입력을 변경하지 않거나 추가 키를 더하여 전달하는 방법입니다. 막약 `RunnablePassthrough()`가 단독으로 호출되면 단순히 입력을 받아 그대로 다음 단계에 전달합니다.
- `RunnablePassthrough.assign(...)`: 입력을 받아 assign 함수에 전달된 인수를 추가합니다.

### RunnablePassthrough

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("다음 문장을 영어로 번역하세요: {input}")
llm = ChatOpenAI(temperature=0)

chain = prompt | llm

In [25]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"input": "안녕하세요"})  # 입력을 그대로 전달

{'input': '안녕하세요'}

아래는 `RunnablePassthrough()` 로 체인을 생성하는 예제입니다.

In [26]:
runnable_chain = {
    "input": RunnablePassthrough()
} | chain  # RunnablePassthrough() 를 사용해서 invoke() 때 키:값 형식이 아니라 값만 받을 수 있게 하는 문법을 많이 사용한다.
runnable_chain.invoke("안녕하세요")

AIMessage(content='Hello', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 25, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3da8d110-3ad2-4015-8e79-75e01a88ab54-0', usage_metadata={'input_tokens': 25, 'output_tokens': 1, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough.assign()` 로 체인을 생성하는 예제입니다.  
assign() 메서드를 활용하면 입력의 일부 요소를 추출하거나, 입력 데이터를 수정하여 다른 구조로 매핑할 수 있습니다.  
예를 들어, JSON 데이터의 특정 필드만을 전달하거나 입력 데이터의 구조를 변경하는 작업에서 유용하게 사용할 수 있습니다.  
용도: 데이터 필터링 또는 재구조화가 필요한 경우에 사용됩니다.  

In [27]:
RunnablePassthrough().assign(
    new_input=lambda x: x["input"] + " 좋은 하루 되세요!"
).invoke({"input": "안녕하세요."})

{'input': '안녕하세요.', 'new_input': '안녕하세요. 좋은 하루 되세요!'}

### RunnableParallel

- RunnableParallel은 여러 Runnable 객체를 동시에 실행하여 병렬 처리합니다. 
- 각 Runnable에 입력 데이터를 전달하고, 그 결과를 동시에 반환합니다.
- 여러 작업을 병렬로 처리하여 성능을 높이려는 경우 유용합니다. 예를 들어, 여러 API 호출을 병렬로 실행하거나, 여러 모델을 동시에 호출할 때 사용할 수 있습니다.  

용도: 작업을 병렬로 처리하여 효율성을 높이고 시간을 절약하려는 경우에 적합합니다.

In [28]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough().assign(
        new_input=lambda x: x["input"] + " 좋은 하루 되세요!"
    ),
    modified=lambda x: x["input"] + " 반갑습니다!",
)
runnable.invoke({"input": "안녕하세요."})

{'passed': {'input': '안녕하세요.'},
 'extra': {'input': '안녕하세요.', 'new_input': '안녕하세요. 좋은 하루 되세요!'},
 'modified': '안녕하세요. 반갑습니다!'}

Chain에 RunnableParallel 를 사용하는 예제입니다.

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(temperature=0)

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 면적은?")
    | llm
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)

combined_chain.invoke("Korea")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 14, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5bb52f7-88d8-4dd2-a173-17acca800812-0', usage_metadata={'input_tokens': 14, 'output_tokens': 5, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 15, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens

### RunnableLambda

- RunnableLambda 를 사용하여 사용자 정의 함수를 맵핑할 수 있습니다. 
- **매개변수가 하나** 일때만 사용합니다. 여러개를 받으려면 dictionary 타입으로 래핑하여 사용하는 방법을 구현 할 수 있습니다.

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


# 왜 굳이 없어도 되는 매개변수를 지정했는가? invoke 함수에 정의된 매개변수가 사용자가 정의한 함수에 들어오긴 들어와야 하는 구조이기 때문에 변수가 있어야 한다.
def get_today(a):
    print(f"입력받은 변수 a의 값: {a}")
    print(f"입력받은 n의 값: {a['n']}")
    return datetime.now().strftime("%m월 %d일")


# 오늘 날짜 출력
# get_today(None)

In [18]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

template = """
{today} 이 생일인 한국이 유명인 {n} 명을 나열해주세요.
양식은 [FORMAT]을 참고하여 표기해주세요.

# FORMAT
- 유명인 이름:
- 생일 날짜:
"""

prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke(3)
print(answer)

- 유명인 이름: 이승기  
- 생일 날짜: 1987년 11월 11일  

- 유명인 이름: 박정현  
- 생일 날짜: 1976년 11월 11일  

- 유명인 이름: 김소현  
- 생일 날짜: 1999년 11월 11일  


`itemgetter` 를 사용하여 특정 키를 추출할 수 있습니다.

In [22]:
from operator import itemgetter

chain = (
    {"today": RunnableLambda(get_today), "n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke({"n": 3})
print(answer)

입력받은 변수 a의 값: {'n': 3}
입력받은 n의 값: 3
- 유명인 이름: 이승기  
- 생일 날짜: 1987년 11월 11일  

- 유명인 이름: 박정현  
- 생일 날짜: 1976년 11월 11일  

- 유명인 이름: 김소현  
- 생일 날짜: 1999년 11월 11일  


`_multiple_function()` 예시입니다. 다중매개변수를 받는 방법입니다.

In [34]:
def number_function(num: int):
    return num


def _multiple_num_function(num1, num2):
    return num1 + num2


def multiple_num_function(_dict):
    return _multiple_num_function(_dict["num1"], _dict["num2"])

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("{a} * {b} 는 무엇인가요?")
llm = ChatOpenAI(temperature=0)

chain = (
    {
        "a": itemgetter("num1") | RunnableLambda(number_function),
        "b": {"num1": itemgetter("num1"), "num2": itemgetter("num2")}
        | RunnableLambda(multiple_num_function),
    }
    | prompt
    | llm
)

answer = chain.invoke({"num1": 2, "num2": 3})
print(answer.content)

2
2 * 5는 10 입니다.
